## Code to Access Data in most recent AkWarm Energy Library

In [5]:
from importlib import reload
from io import BytesIO
import gzip
import xml.etree.ElementTree as ET
import requests

In [9]:
def get_akwarm_lib_xml():
    """Returns the root XML ElementTree of the most current AkWarm Energy Library.
    Prints the name of that library.
    """
    resp = requests.get('https://analysisnorth.com/AkWarm/update_combined/Library_Info.txt')
    cur_lib_name = resp.text.splitlines()[-1].split('\t')[0]
    resp = requests.get(f'https://analysisnorth.com/AkWarm/update_combined/{cur_lib_name}')
    res = [ x ^ 30 for x in resp.content]
    file_res = BytesIO(bytes(res))
    del res   # to save memory
    lib = gzip.GzipFile(mode='r', fileobj=file_res).read()
    root = ET.fromstring(lib.decode('utf-8'))
    return root, cur_lib_name

In [10]:
def write_city_list():
    """Writes into a file the list of Cities from the most current AkWarm Energy Library.
    File name is 'city-list.txt' and is placed in the current directory.  One city per line,
    sorted alphabetically.
    """
    root, _ = get_akwarm_lib_xml()
    cities = root.find("./item/key/string[.='City']/../../value/ArrayOfCity")
    city_list = []
    for city in cities:
        city_name = city.find("Name").text
        if city.find('Active').text == 'true':
            city_list.append(city_name)
        else:
            print('Inactive:', city_name)

    city_list.sort()
    with open('city-list.txt', 'w') as fout:
        for city in city_list:
            print(city, file=fout)

In [11]:
def get_akwarm_city_data():
    """Returns a dictionary keyed on City Name with the dictionary value being a dictionary
    of key fields and values from the City table in the most recent AkWarm Energy Library.
    Makes an attempt to convert the field value to a float.
    """
    root, lib_name = get_akwarm_lib_xml()
    city_data = {}
    targets = ('Oil1Price', 'Oil2Price')
    cities = root.find("./item/key/string[.='City']/../../value/ArrayOfCity")
    for city in cities:
        fields = {}    
        for fld in targets:
            val = city.find(fld).text
            try:
                val = float(city.find(fld).text)
            except:
                pass
            fields[fld] = val
        city_data[city.find("Name").text] = fields

    return city_data, cur_lib_name

In [12]:
write_city_list()

Inactive: Mertarvilk


In [17]:
city_info, lib_name = get_akwarm_city_data()
print(lib_name)
city_info

2021-08-31.lib


{'Anchorage': {'Oil1Price': 3.36, 'Oil2Price': None},
 'Adak': {'Oil1Price': 4.73, 'Oil2Price': 4.73},
 'Akutan': {'Oil1Price': None, 'Oil2Price': 2.55},
 'Allakaket': {'Oil1Price': 7.0, 'Oil2Price': 7.0},
 'Angoon': {'Oil1Price': 4.15, 'Oil2Price': None},
 'Aniak': {'Oil1Price': 6.05, 'Oil2Price': None},
 'Arctic Village': {'Oil1Price': 12.0, 'Oil2Price': None},
 'Atka': {'Oil1Price': 7.5, 'Oil2Price': None},
 'Auke Bay': {'Oil1Price': 3.1, 'Oil2Price': 3.12},
 'Barrow/Utqiavik': {'Oil1Price': None, 'Oil2Price': None},
 'Bethel': {'Oil1Price': 3.73, 'Oil2Price': None},
 'Bettles': {'Oil1Price': 7.45, 'Oil2Price': None},
 'Delta Junction': {'Oil1Price': 2.93, 'Oil2Price': None},
 'Big Lake': {'Oil1Price': 2.09, 'Oil2Price': None},
 'Cantwell': {'Oil1Price': 3.46, 'Oil2Price': None},
 'Central': {'Oil1Price': 3.6, 'Oil2Price': None},
 'Chalkyitsik': {'Oil1Price': 8.0, 'Oil2Price': None},
 'Chickaloon': {'Oil1Price': 2.8, 'Oil2Price': None},
 'Chicken': {'Oil1Price': 3.1, 'Oil2Price': No

In [20]:
import gspread
from config import spreadsheet_creds_file
gc = gspread.service_account(filename=spreadsheet_creds_file)
wb = gc.open_by_key('1FE5Lri1v_I6ZRxgfikdsU72NWKfr7eJ2OYjcbpsjaY4')

In [22]:
vals = wb.worksheet('Customers').get_all_values()

In [23]:
vals[3]

['city',
 'customer',
 'cust_email',
 'anthc_emails',
 'sensor_id',
 'btu_mult',
 'fuel_categ',
 'pct_fuel_billed',
 'util_akw_disc',
 'util_fuel_override',
 'akwarm_city',
 'cust_fuel_disc',
 'cust_fuel_override',
 'feas_g_01',
 'feas_g_02',
 'feas_g_03',
 'feas_g_04',
 'feas_g_05',
 'feas_g_06',
 'feas_g_07',
 'feas_g_08',
 'feas_g_09',
 'feas_g_10',
 'feas_g_11',
 'feas_g_12']

In [24]:
import data_util
reload(data_util)

<module 'data_util' from '/home/tabb99/heat-recovery-billing/data_util.py'>

In [25]:
data_util.get_akwarm_city_data()

({'Anchorage': {'Oil1Price': 3.36, 'Oil2Price': None},
  'Adak': {'Oil1Price': 4.73, 'Oil2Price': 4.73},
  'Akutan': {'Oil1Price': None, 'Oil2Price': 2.55},
  'Allakaket': {'Oil1Price': 7.0, 'Oil2Price': 7.0},
  'Angoon': {'Oil1Price': 4.15, 'Oil2Price': None},
  'Aniak': {'Oil1Price': 6.05, 'Oil2Price': None},
  'Arctic Village': {'Oil1Price': 12.0, 'Oil2Price': None},
  'Atka': {'Oil1Price': 7.5, 'Oil2Price': None},
  'Auke Bay': {'Oil1Price': 3.1, 'Oil2Price': 3.12},
  'Barrow/Utqiavik': {'Oil1Price': None, 'Oil2Price': None},
  'Bethel': {'Oil1Price': 3.73, 'Oil2Price': None},
  'Bettles': {'Oil1Price': 7.45, 'Oil2Price': None},
  'Delta Junction': {'Oil1Price': 2.93, 'Oil2Price': None},
  'Big Lake': {'Oil1Price': 2.09, 'Oil2Price': None},
  'Cantwell': {'Oil1Price': 3.46, 'Oil2Price': None},
  'Central': {'Oil1Price': 3.6, 'Oil2Price': None},
  'Chalkyitsik': {'Oil1Price': 8.0, 'Oil2Price': None},
  'Chickaloon': {'Oil1Price': 2.8, 'Oil2Price': None},
  'Chicken': {'Oil1Price': 3

In [23]:
data_util.utility_fuel_prices()

{'AVEC Northwest': 3.75, 'AVEC YK': 3.25, 'AVEC Interior': 4.2, 'Other': nan}

In [19]:
data_util.customer_records()

[{'city': 'Chevak',
  'customer': 'Water Plant',
  'cust_email': 'xyz@chevak.us',
  'anthc_emails': 'cocox@anthc.org, gdhodgson@anthc.org, dmmadden1@anthc.org',
  'sensor_id': 'A81758FFFE0565CE_pulseAbs_raw',
  'btu_mult': 1.0,
  'fuel_categ': 'AVEC YK',
  'pct_fuel_billed': 0.3,
  'util_akw_disc': nan,
  'util_fuel_override': nan,
  'akwarm_city': 'Chevak',
  'cust_fuel_disc': 0.0,
  'cust_fuel_override': nan,
  'feas_g_01': 200.0,
  'feas_g_02': 180.0,
  'feas_g_03': 150.0,
  'feas_g_04': 100.0,
  'feas_g_05': 50.0,
  'feas_g_06': 20.0,
  'feas_g_07': 0.0,
  'feas_g_08': 10.0,
  'feas_g_09': 50.0,
  'feas_g_10': 80.0,
  'feas_g_11': 120.0,
  'feas_g_12': 180.0}]

In [12]:
data_util.customer_records()[0].keys()

dict_keys(['city', 'customer', 'cust_email', 'anthc_emails', 'sensor_id', 'btu_mult', 'fuel_categ', 'pct_fuel_billed', 'util_akw_disc', 'util_fuel_override', 'akwarm_city', 'cust_fuel_disc', 'cust_fuel_override', 'feas_g_01', 'feas_g_02', 'feas_g_03', 'feas_g_04', 'feas_g_05', 'feas_g_06', 'feas_g_07', 'feas_g_08', 'feas_g_09', 'feas_g_10', 'feas_g_11', 'feas_g_12'])

In [40]:
from dateutil.parser import parse
parse('Oct 2021').strftime('%Y-%m-%d')

'2021-10-05'